In [1]:
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.datasets import imdb

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
endpoint_name = 'sentiment-analysis-e84389e8-9803-ef3a-6ead-ddbba24c9ec5-DEV'

class Predictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer, 
                         deserializer=json_deserializer, content_type='application/json')

sentiment_predictor = Predictor(endpoint_name, sagemaker_session=sagemaker_session)

In [2]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

25000 train sequences
25000 test sequences
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [3]:
regex = re.compile(r'^[\?\s]+')
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [4]:
review_index = 10
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_test[review_index]])
print(regex.sub('', decoded_review))

inspired by hitchcock's strangers on a train concept of two men swapping murders in exchange for getting rid of the two people messing up their lives throw ? from the train is an original and very inventive comedy take on the idea it's a credit to danny devito that he both wrote and starred in this minor comedy gem br br anne ramsey is the mother who inspires the film's title and it's understandable why she gets under the skin of danny devito with her sharp tongue and relentlessly putting him down for any minor ? billy crystal is the writer who's wife has stolen his book idea and is now being ? as a great new author even appearing on the oprah show to in ? he should be enjoying thus devito gets the idea of swapping murders to rid themselves of these nuisance factors br br of course everything and anything can happen when writer carl reiner lets his imagination roam with ? ideas for how the plot develops and it's amusing all the way through providing plenty of laughs and chuckles along 

In [5]:
results = sentiment_predictor.predict(x_test[review_index])['predictions'][0][0]

def get_sentiment(score):
    return 'positive' if score > 0.5 else 'negative' 

print('Labeled sentiment for this review is {}, predicted sentiment is {}'.format(get_sentiment(y_test[review_index]), 
                                                                                  get_sentiment(results)))

Labeled sentiment for this review is positive, predicted sentiment is positive
